In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchinfo import summary
import matplotlib.pyplot as plt
import torch.optim as optim

In [24]:
# mean and sd for images size 299,299
mean = [0.1854, 0.1854, 0.1855]
std = [0.2005, 0.2005, 0.2005]

batch_size = 32
epochs = 10
lr = 0.0001

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
data_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [19]:
# Load the datasets
train_dataset = datasets.ImageFolder(root="../data/Training", transform=data_transform)
test_dataset = datasets.ImageFolder(root="../data/Testing", transform=data_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
# Initialize Inception V3 Model
model = models.inception_v3(weights=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, len(train_dataset.classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# summary(model, input_size=(1, 3, 299, 299))

Layer (type:depth-idx)                   Output Shape              Param #
Inception3                               [1, 4]                    3,326,696
├─BasicConv2d: 1-1                       [1, 32, 149, 149]         --
│    └─Conv2d: 2-1                       [1, 32, 149, 149]         864
│    └─BatchNorm2d: 2-2                  [1, 32, 149, 149]         64
├─BasicConv2d: 1-2                       [1, 32, 147, 147]         --
│    └─Conv2d: 2-3                       [1, 32, 147, 147]         9,216
│    └─BatchNorm2d: 2-4                  [1, 32, 147, 147]         64
├─BasicConv2d: 1-3                       [1, 64, 147, 147]         --
│    └─Conv2d: 2-5                       [1, 64, 147, 147]         18,432
│    └─BatchNorm2d: 2-6                  [1, 64, 147, 147]         128
├─MaxPool2d: 1-4                         [1, 64, 73, 73]           --
├─BasicConv2d: 1-5                       [1, 80, 73, 73]           --
│    └─Conv2d: 2-7                       [1, 80, 73, 73]           5,

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Clear previous gradients
            optimizer.zero_grad()
            
            # Inception v3 has two outputs: primary and auxiliary
            outputs, aux_outputs = model(images)  
            # calculate weighted loss
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux_outputs, labels)
            loss = loss1 + 0.4 * loss2
            
            # Backpropagate and update weights
            loss.backward()  
            optimizer.step()
            
            running_loss += loss.item()
            
            # Get predicted class
            _, predicted = outputs.max(1)  
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {accuracy:.2f}%")

In [30]:
# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=epochs)

Epoch 1/10 - Loss: 0.4825 - Accuracy: 91.40%
Epoch 2/10 - Loss: 0.0679 - Accuracy: 98.48%
Epoch 3/10 - Loss: 0.0393 - Accuracy: 99.23%
Epoch 4/10 - Loss: 0.0219 - Accuracy: 99.53%
Epoch 5/10 - Loss: 0.0222 - Accuracy: 99.60%
Epoch 6/10 - Loss: 0.0234 - Accuracy: 99.46%


KeyboardInterrupt: 